# Handout #4: Unsupervised Learning (hierarchical clustering)

## Problem 5

Load needed libraries

In [2]:
import numpy as np

### Load the *dataset*

Our group is '01'

In [3]:
group = '01'  # BERNAT LADARIA, Lluís
ds = 5

Load vectors from file

In [4]:
data = np.loadtxt('ds'+group+str(ds)+'.txt')

Split data and labels

In [43]:
X = data[:, 0:2]
y = data[:, 2]

In [44]:
X.shape

(1000, 2)

In [59]:
y.shape

(1000,)

### Cluster the data

Import necessary functions and classes

In [16]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import v_measure_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

In [16]:
import itertools

Create the options lists to use

In [12]:
linkage_list = ['complete', 'average', 'ward']

In [13]:
n_clusters_list = [2, 3, 4, 5]

In [14]:
affinity_list = ['euclidean']

Prepare a models list and populate it with all the trials

In [64]:
models_list = list()

In [65]:
for linkage, n_clusters, affinity in itertools.product(linkage_list, n_clusters_list, affinity_list):
    model = AgglomerativeClustering(linkage=linkage, n_clusters=n_clusters, affinity=affinity)
    model.fit(X)
    
    entry = {
        'linkage': linkage,
        'n_clusters': n_clusters,
        'affinity': affinity,
        'model': model, 
        'v_measure_score': v_measure_score(y, model.labels_)
    }
    models_list.append(entry)

In [66]:
models_list

[{'linkage': 'complete',
  'n_clusters': 2,
  'affinity': 'euclidean',
  'model': AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                          connectivity=None, distance_threshold=None,
                          linkage='complete', memory=None, n_clusters=2),
  'v_measure_score': 0.6879066626694416},
 {'linkage': 'complete',
  'n_clusters': 3,
  'affinity': 'euclidean',
  'model': AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                          connectivity=None, distance_threshold=None,
                          linkage='complete', memory=None, n_clusters=3),
  'v_measure_score': 0.943473573294366},
 {'linkage': 'complete',
  'n_clusters': 4,
  'affinity': 'euclidean',
  'model': AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                          connectivity=None, distance_threshold=None,
                          linkage='complete', memory=None, n_clusters=4),
  'v_measure_score': 0.

## Problem 6